<a href="https://colab.research.google.com/github/haminhtien99/re3-pytorch/blob/master/home.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/Re3-Object-Tracking/re3-pytorch
# !git clone https://github.com/danielgordon10/re3-pytorch

/content/drive/MyDrive/Re3-Object-Tracking/re3-pytorch


### make file npy

In [ ]:
import cv2
import numpy as np
import glob
import xml.etree.ElementTree as ET
import time
import random
import os
import sys


In [ ]:
from re3_utils.util import drawing
from re3_utils.util.im_util import get_image_size


In [ ]:
import shutil

In [ ]:
%cd /content/drive/MyDrive/Re3-Object-Tracking/re3-pytorch/demo/data

In [ ]:
folder = "/content/drive/MyDrive/Re3-Object-Tracking/re3-pytorch/demo/data/"
jpg_files = [f for f in os.listdir(folder) if f.endswith(".jpg")]
jpg_files.sort()
num_train = len(jpg_files)//3 * 2

img_train = jpg_files[:num_train]
img_test = jpg_files[-num_train:]
print(img_train)
for file_name in img_train:
    source_path = os.path.join(folder, file_name)
    dest_path = os.path.join(folder, "image", "train")
    if not os.path.exists(dest_path):
        os.makedirs(dest_path)
    shutil.copy(source_path, dest_path)

['00000169.jpg', '00000170.jpg', '00000171.jpg', '00000172.jpg', '00000173.jpg', '00000174.jpg', '00000175.jpg', '00000176.jpg', '00000177.jpg', '00000178.jpg', '00000179.jpg', '00000180.jpg', '00000181.jpg', '00000182.jpg', '00000183.jpg', '00000184.jpg', '00000185.jpg', '00000186.jpg', '00000187.jpg', '00000188.jpg', '00000189.jpg', '00000190.jpg', '00000191.jpg', '00000192.jpg', '00000193.jpg', '00000194.jpg', '00000195.jpg', '00000196.jpg', '00000197.jpg', '00000198.jpg', '00000199.jpg', '00000200.jpg', '00000201.jpg', '00000202.jpg', '00000203.jpg', '00000204.jpg', '00000205.jpg', '00000206.jpg', '00000207.jpg', '00000208.jpg', '00000209.jpg', '00000210.jpg', '00000211.jpg', '00000212.jpg', '00000213.jpg', '00000214.jpg', '00000215.jpg', '00000216.jpg', '00000217.jpg', '00000218.jpg', '00000219.jpg', '00000220.jpg', '00000221.jpg', '00000222.jpg', '00000223.jpg', '00000224.jpg', '00000225.jpg', '00000226.jpg', '00000227.jpg', '00000228.jpg', '00000229.jpg', '00000230.jpg', '000002

In [ ]:
for file_name in img_test:
    source_path = os.path.join(folder, file_name)
    dest_path = os.path.join(folder, "image", "test")
    if not os.path.exists(dest_path):
        os.makedirs(dest_path)
    shutil.copy(source_path, dest_path)

In [ ]:
labels_txt = folder +"labels.txt"
with open(labels_txt, 'r') as file:
    lines = file.read().splitlines()
bboxes = []
for line in lines[:num_train]:
    # print(line)
    line = [int(float(number)) for number in line.split()]
    imNum =  line[0]
    videoId = 0
    trackId = 0
    classId = 0
    occl = 0
    bbox = [line[1], line[2], line[3], line[4], videoId, trackId, imNum, classId, occl]
    bboxes.append(bbox)
bboxes = np.array(bboxes)
# print(bboxes)
dest_path = os.path.join(folder, "labels", "train")
if not os.path.exists(dest_path):
    os.makedirs(dest_path)
np.save(dest_path + "/labels.npy", bboxes)


In [ ]:
bboxes = []
for line in lines[-num_train:]:
    # print(line)
    line = [int(float(number)) for number in line.split()]
    imNum =  line[0]
    videoId = 0
    trackId = 0
    classId = 0
    occl = 0
    bbox = [line[1], line[2], line[3], line[4], videoId, trackId, imNum, classId, occl]
    bboxes.append(bbox)
bboxes = np.array(bboxes)
dest_path = os.path.join(folder, "labels", "test")
if not os.path.exists(dest_path):
    os.makedirs(dest_path)
np.save(dest_path + "/labels.npy", bboxes)

### af


In [ ]:
!python3 training/get_datasets.py

/content/drive/MyDrive/Re3-Object-Tracking/re3-pytorch/../../../../Datasets/Imagenet_Video/provided_small
/content/drive/MyDrive/Re3-Object-Tracking/re3-pytorch/training/datasets/imagenet_video


In [ ]:
%cd /content/drive/MyDrive/Re3-Object-Tracking/re3-pytorch/training
!python unrolled_solver.py -rtc -n 2 -b 64

/content/drive/MyDrive/Re3-Object-Tracking/re3-pytorch/training
2023-10-04 09:12:23.452197: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-04 09:12:24.606181: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Instructions for updating:
non-resource variables are not supported in the long term
Traceback (most recent call last):
  File "/content/drive/MyDrive/Re3-Object-Tracking/re3-pytorch/training/unrolled_solver.py", line 295, in <module>
    main(args)
  File "/content/drive/MyDrive/Re3-Object-Tracking/re3-pytorch/training/unrolled_solver.py", line 63, in main
    data_loader = pt_dataset.get_data_loader(num_unrolls, batch_size, args.num_threads)
  File "/content/drive/MyDrive/Re3-Object-Tracking/r

In [ ]:
import glob
import os.path
import sys

import cv2

basedir = "/content/drive/MyDrive/Re3-Object-Tracking/re3-pytorch/demo"
sys.path.append(os.path.abspath(os.path.join(basedir, os.path.pardir)))
from tracker import re3_tracker

if not os.path.exists(os.path.join(basedir, "data")):
    import tarfile

    tar = tarfile.open(os.path.join(basedir, "data.tar.gz"))
    tar.extractall(path=basedir)

# cv2.namedWindow("Image", cv2.WINDOW_NORMAL)
# cv2.resizeWindow("Image", 640, 480)
tracker = re3_tracker.Re3Tracker()
image_paths = sorted(glob.glob(os.path.join(basedir, "data", "*.jpg")))
initial_bbox = [175, 154, 251, 229]

#initial_bbox = [0, 0, 227, 227]
tracker.track("ball", image_paths[0], initial_bbox)
for image_path in image_paths:
    image = cv2.imread(image_path)
    # Tracker expects RGB, but opencv loads BGR.
    imageRGB = image[:, :, ::-1]
    bbox = tracker.track("ball", imageRGB)
    print(bbox)
    # cv2.rectangle(image, (int(bbox[0]), int(bbox[1])), (int(bbox[2]), int(bbox[3])), [0, 0, 255], 2)
    # cv2.imshow("Image", image)
    # cv2.waitKey(1)

restoring from /content/drive/MyDrive/Re3-Object-Tracking/re3-pytorch/logs/checkpoints
No checkpoints found


RuntimeError: ignored